In [1]:
# !pip install transformers datasets tokenizers seqeval -q

In [2]:
import re
import numpy as np 
import pandas as pd
import json
from num2words import num2words

from datasets import load_dataset, load_metric
from transformers import BertTokenizerFast, DataCollatorForTokenClassification, AutoModelForTokenClassification, TrainingArguments, Trainer, pipeline 

from sklearn.model_selection import train_test_split

In [ ]:
random_seed = 1
transformers.set_seed(seed=random_seed)


## Transform NLP Data

In [3]:
# df_nlp = pd.read_json('data/nlp.jsonl', lines=True)
# df_nlp = pd.read_json('../data/nlp.jsonl', lines=True).iloc[:1000]
df_nlp = pd.read_json('../data/nlp.jsonl', lines=True).sample(n=1000)

df_nlp['tokens'] = df_nlp['transcript'].apply(lambda x: re.findall(r'[.?]|\w+', x))
df_nlp['tool'] = df_nlp['tool'].apply(lambda x: re.findall(r'[.?]|\w+', x))
df_nlp['heading'] = df_nlp['heading'].apply(lambda x: list(map(lambda y: num2words(int(y)), list(str(x).zfill(3)))))
df_nlp['target'] = df_nlp['target'].apply(lambda x: re.findall(r'[.?]|\w+', x))

def map_tags(lst, items:str):
	dic = {}
	begin = ''
	inbet = ''
	if items == 'tool':
		begin = 1 #'B-TOL'
		inbet = 2 #'I-TOL'
	elif items == 'heading':
		begin = 3 #'B-HDG'
		inbet = 4 #'I-HDG'
	elif items == 'target':
		begin = 5 #'B-TAR'
		inbet = 6 # 'I-TAR'
	dic[lst[0]] = begin
	for i in range(1, len(lst)):
		dic[lst[i]] = inbet
	return dic

df_nlp['tool'] = df_nlp['tool'].apply(lambda x: map_tags(x, 'tool'))
df_nlp['heading'] = df_nlp['heading'].apply(lambda x: map_tags(x, 'heading'))
df_nlp['target'] = df_nlp['target'].apply(lambda x: map_tags(x, 'target'))
df_nlp = df_nlp[['key', 'tokens', 'tool', 'heading', 'target']]
df_nlp = df_nlp.rename(columns={'key':'id'})

def map_num(row):
	curr_dict = row['tool'].copy()
	curr_dict.update(row['heading'])
	curr_dict.update(row['target'])	
	return [0 if curr_dict.get(i) is None else curr_dict.get(i) for i in row['tokens']]

df_nlp['ner_tags'] = df_nlp.apply(map_num, axis=1)
df_nlp

,id,tokens,tool,heading,target,ner_tags
0,0,"[Turret, prepare, to, deploy, electromagnetic,...","{'electromagnetic': 1, 'pulse': 2}","{'zero': 3, 'six': 4, 'five': 4}","{'grey': 5, 'and': 6, 'white': 6, 'fighter': 6...","[0, 0, 0, 0, 1, 2, 0, 0, 3, 4, 4, 0, 0, 5, 6, ..."
1,1,"[Engage, yellow, drone, with, surface, to, air...","{'surface': 1, 'to': 2, 'air': 2, 'missiles': 2}","{'two': 3, 'three': 4, 'five': 4}","{'yellow': 5, 'drone': 6}","[0, 5, 6, 0, 1, 2, 2, 2, 0, 3, 4, 4, 0]"
2,2,"[Control, to, turrets, deploy, electromagnetic...","{'electromagnetic': 1, 'pulse': 2}","{'one': 4, 'zero': 4}","{'blue': 5, 'and': 6, 'red': 6, 'fighter': 6, ...","[0, 0, 0, 0, 1, 2, 0, 4, 4, 4, 0, 0, 5, 6, 6, ..."
3,3,"[Alfa, Echo, Mike, Papa, deploy, EMP, tool, he...",{'EMP': 1},"{'zero': 3, 'eight': 4, 'five': 4}","{'purple': 5, 'red': 6, 'and': 6, 'silver': 6,...","[0, 0, 0, 0, 0, 1, 0, 0, 3, 4, 4, 0, 5, 6, 6, ..."
4,4,"[Engage, the, grey, black, and, green, fighter...","{'machine': 1, 'gun': 2}","{'zero': 3, 'nine': 4, 'five': 4}","{'grey': 5, 'black': 6, 'and': 6, 'green': 6, ...","[0, 0, 5, 6, 6, 6, 6, 6, 0, 0, 1, 2, 0, 3, 0, ..."
...,...,...,...,...,...,...
995,995,"[Turret, Bravo, lock, onto, the, brown, purple...","{'machine': 1, 'gun': 2}","{'two': 3, 'four': 4, 'zero': 4}","{'brown': 5, 'purple': 6, 'and': 6, 'red': 6, ...","[0, 0, 0, 0, 0, 5, 6, 6, 6, 6, 6, 0, 3, 4, 4, ..."
996,996,"[Turrets, Foxtrot, and, Golf, adjust, heading,...","{'surface': 1, 'to': 2, 'air': 2, 'missiles': 2}","{'one': 3, 'four': 4, 'zero': 4}","{'white': 5, 'and': 6, 'green': 6, 'cargo': 6,...","[0, 0, 6, 0, 0, 0, 2, 3, 4, 4, 0, 0, 0, 5, 6, ..."
997,997,"[Control, to, air, defense, turrets, prepare, ...","{'anti': 1, 'air': 2, 'artillery': 2}","{'zero': 4, 'five': 4}","{'black': 5, 'yellow': 6, 'and': 6, 'red': 6, ...","[0, 0, 2, 0, 0, 0, 0, 0, 0, 5, 6, 6, 6, 6, 6, ..."
998,998,"[Turret, Alpha, engage, the, target, orange, a...","{'anti': 1, 'air': 2, 'artillery': 2}","{'zero': 3, 'one': 4, 'five': 4}","{'orange': 5, 'and': 6, 'green': 6, 'cargo': 6...","[0, 0, 0, 0, 0, 5, 6, 6, 6, 6, 0, 0, 3, 4, 4, ..."


In [4]:
label_list = ['O', 'B-TOL', 'I-TOL', 'B-HDG', 'I-HDG', 'B-TAR', 'I-TAR']

## Load NLP Data

In [5]:
train, test = train_test_split(df_nlp, test_size=0.2, random_state=random_seed)
train, val = train_test_split(train, test_size=0.125, random_state=random_seed) # 0.125 x 0.8 = 0.1
train.shape, val.shape, test.shape

((700, 6), (100, 6), (200, 6))

In [6]:
train.to_json('../data/nlp_train.jsonl', orient='records', lines=True)
test.to_json('../data/nlp_test.jsonl', orient='records', lines=True)
val.to_json('../data/nlp_val.jsonl', orient='records', lines=True)

In [7]:
_nlp_data = {
	'train' : '../data/nlp_train.jsonl',
	'validation' : '../data/nlp_val.jsonl',
	'test' : '../data/nlp_test.jsonl'
}

nlp_data =  load_dataset('json', data_files=_nlp_data) 
nlp_data

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'tool', 'heading', 'target', 'ner_tags'],
        num_rows: 700
    })
    validation: Dataset({
        features: ['id', 'tokens', 'tool', 'heading', 'target', 'ner_tags'],
        num_rows: 100
    })
    test: Dataset({
        features: ['id', 'tokens', 'tool', 'heading', 'target', 'ner_tags'],
        num_rows: 200
    })
})

## Tokenization

In [8]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased") 

c:\Users\Keck\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
def tokenize_and_align_labels(examples, label_all_tokens=True): 
    """
    Function to tokenize and align labels with respect to the tokens. This function is specifically designed for
    Named Entity Recognition (NER) tasks where alignment of the labels is necessary after tokenization.

    Parameters:
    examples (dict): A dictionary containing the tokens and the corresponding NER tags.
                     - "tokens": list of words in a sentence.
                     - "ner_tags": list of corresponding entity tags for each word.
                     
    label_all_tokens (bool): A flag to indicate whether all tokens should have labels. 
                             If False, only the first token of a word will have a label, 
                             the other tokens (subwords) corresponding to the same word will be assigned -100.

    Returns:
    tokenized_inputs (dict): A dictionary containing the tokenized inputs and the corresponding labels aligned with the tokens.
    """
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True) 
    labels = [] 
    for i, label in enumerate(examples["ner_tags"]): 
        word_ids = tokenized_inputs.word_ids(batch_index=i) 
        # word_ids() => Return a list mapping the tokens
        # to their actual word in the initial sentence.
        # It Returns a list indicating the word corresponding to each token. 
        previous_word_idx = None 
        label_ids = []
        # Special tokens like `<s>` and `<\s>` are originally mapped to None 
        # We need to set the label to -100 so they are automatically ignored in the loss function.
        for word_idx in word_ids: 
            if word_idx is None: 
                # set –100 as the label for these special tokens
                label_ids.append(-100)
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            elif word_idx != previous_word_idx:
                # if current word_idx is != prev then its the most regular case
                # and add the corresponding token                 
                label_ids.append(label[word_idx]) 
            else: 
                # to take care of sub-words which have the same word_idx
                # set -100 as well for them, but only if label_all_tokens == False
                label_ids.append(label[word_idx] if label_all_tokens else -100) 
                # mask the subword representations after the first subword
                 
            previous_word_idx = word_idx 
        labels.append(label_ids) 
    tokenized_inputs["labels"] = labels 
    return tokenized_inputs 

### Add Labels

In [10]:
tokenized_datasets = nlp_data.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/700 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

# Model Training

In [11]:
def compute_metrics(eval_preds): 
    """
    Function to compute the evaluation metrics for Named Entity Recognition (NER) tasks.
    The function computes precision, recall, F1 score and accuracy.

    Parameters:
    eval_preds (tuple): A tuple containing the predicted logits and the true labels.

    Returns:
    A dictionary containing the precision, recall, F1 score and accuracy.
    """
    pred_logits, labels = eval_preds 
    
    pred_logits = np.argmax(pred_logits, axis=2) 
    # the logits and the probabilities are in the same order,
    # so we don’t need to apply the softmax
    
    # We remove all the values where the label is -100
    predictions = [ 
        [label_list[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100] 
        for prediction, label in zip(pred_logits, labels) 
    ] 
    
    true_labels = [ 
      [label_list[l] for (eval_preds, l) in zip(prediction, label) if l != -100] 
       for prediction, label in zip(pred_logits, labels) 
   ] 
    results = metric.compute(predictions=predictions, references=true_labels) 
    return { 
   "precision": results["overall_precision"], 
   "recall": results["overall_recall"], 
   "f1": results["overall_f1"], 
  "accuracy": results["overall_accuracy"], 
  } 

In [12]:
model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels=7)

args = TrainingArguments( 
	"test-ner",
	evaluation_strategy = "epoch", 
	learning_rate=2e-5, 
	per_device_train_batch_size=16, 
	per_device_eval_batch_size=16, 
	num_train_epochs=3, 
	weight_decay=0.01, 
) 

data_collator = DataCollatorForTokenClassification(tokenizer) 

metric = load_metric("seqeval") 

trainer = Trainer( 
    model, 
	args, 
	train_dataset=tokenized_datasets["train"], 
	eval_dataset=tokenized_datasets["validation"], 
	data_collator=data_collator, 
	tokenizer=tokenizer, 
	compute_metrics=compute_metrics 
) 

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Keck\AppData\Local\Temp\ipykernel_25612\2808789182.py:15: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
c:\Users\Keck\AppData\Local\Programs\Python\Python310\lib\site-packages\datasets\load.py:759: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_r

In [13]:
trainer.train() 

c:\Users\Keck\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/132 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.09678119421005249, 'eval_precision': 0.9716713881019831, 'eval_recall': 0.9346049046321526, 'eval_f1': 0.9527777777777778, 'eval_accuracy': 0.9813084112149533, 'eval_runtime': 4.3918, 'eval_samples_per_second': 22.77, 'eval_steps_per_second': 1.594, 'epoch': 1.0}


  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.037862084805965424, 'eval_precision': 0.9806094182825484, 'eval_recall': 0.9645776566757494, 'eval_f1': 0.9725274725274725, 'eval_accuracy': 0.9878504672897196, 'eval_runtime': 4.2784, 'eval_samples_per_second': 23.373, 'eval_steps_per_second': 1.636, 'epoch': 2.0}


  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.03078080154955387, 'eval_precision': 0.9836512261580381, 'eval_recall': 0.9836512261580381, 'eval_f1': 0.9836512261580381, 'eval_accuracy': 0.9920560747663552, 'eval_runtime': 4.2046, 'eval_samples_per_second': 23.784, 'eval_steps_per_second': 1.665, 'epoch': 3.0}
{'train_runtime': 347.5771, 'train_samples_per_second': 6.042, 'train_steps_per_second': 0.38, 'train_loss': 0.22773476802941525, 'epoch': 3.0}


TrainOutput(global_step=132, training_loss=0.22773476802941525, metrics={'train_runtime': 347.5771, 'train_samples_per_second': 6.042, 'train_steps_per_second': 0.38, 'train_loss': 0.22773476802941525, 'epoch': 3.0})

## Save Model

In [14]:
model.save_pretrained("ner_model")
tokenizer.save_pretrained("tokenizer")

('tokenizer\\tokenizer_config.json',
 'tokenizer\\special_tokens_map.json',
 'tokenizer\\vocab.txt',
 'tokenizer\\added_tokens.json',
 'tokenizer\\tokenizer.json')

In [15]:
id2label = {
    str(i): label for i,label in enumerate(label_list)
}
label2id = {
    label: str(i) for i,label in enumerate(label_list)
}

config = json.load(open("ner_model/config.json"))
config["id2label"] = id2label
config["label2id"] = label2id

# Model Prediction

In [16]:
json.dump(config, open("ner_model/config.json","w"))

In [17]:
model_fine_tuned = AutoModelForTokenClassification.from_pretrained("ner_model")

In [18]:
nlp = pipeline("ner", model=model_fine_tuned, tokenizer=tokenizer)


example = "Shoot down the car, red it is, as well as white, 1 six two, javelin missiles at the ready"

ner_results = nlp(example)

for i in ner_results:
	print(i)

{'entity': 'I-TAR', 'score': 0.8785443, 'index': 4, 'word': 'car', 'start': 15, 'end': 18}
{'entity': 'B-TAR', 'score': 0.8562962, 'index': 6, 'word': 'red', 'start': 20, 'end': 23}
{'entity': 'B-TAR', 'score': 0.9575715, 'index': 13, 'word': 'white', 'start': 42, 'end': 47}
{'entity': 'B-HDG', 'score': 0.87913495, 'index': 15, 'word': '1', 'start': 49, 'end': 50}
{'entity': 'I-HDG', 'score': 0.9915069, 'index': 16, 'word': 'six', 'start': 51, 'end': 54}
{'entity': 'I-HDG', 'score': 0.9864733, 'index': 17, 'word': 'two', 'start': 55, 'end': 58}
{'entity': 'B-TOL', 'score': 0.7350409, 'index': 19, 'word': 'javelin', 'start': 60, 'end': 67}
{'entity': 'I-TOL', 'score': 0.75939256, 'index': 20, 'word': 'missiles', 'start': 68, 'end': 76}
